# TABLE 2
Performance bounds for FG with standard tuning using high-precision LMI solver.

In [5]:
include("optalg.jl")     # import necessary functions

using SCS, MosekTools, Hypatia    # load alternative solvers

In [3]:
setprecision(BigFloat, 256)

m = big"1"
L = big"100"

# Nesterov's method
α = big"1"/L
β = (√L-√m)/(√L+√m)
η = (√L-√m)/(√L+√m)

α, β

(0.009999999999999999999999999999999999999999999999999999999999999999999999999999995, 0.8181818181818181818181818181818181818181818181818181818181818181818181818181787)

## High-precision solution of sensitivity LMI
Solve for various values of ℓ and compute dual solution to certify error bound on sensitivity γ.

In [25]:
for ℓ in 1:10
    model = solver_helper(α, β, η, m, L, big"1", ℓ; returnsol=false, bal=false, norm="H2", solver=Hypatia, T=BigFloat);
    pv = sqrt(objective_value(model))
    dv = sqrt(dual_objective_value(model))
    err = Float64(abs(pv-dv))
    if ℓ < 10
        println("ℓ = $ℓ : γ = $pv ± $err")
    else
        println("ℓ = $ℓ: γ = $pv ± $err")
    end
    assert_is_solved_and_feasible(model; dual = true)
end

ℓ = 1 : γ = 0.2007655549416654096651702226524048017730848560267265265100558688088339705871765 ± 2.9539593307982505e-31
ℓ = 2 : γ = 0.1859086893723726554494151997267118858192204117530077039663495876388066250951542 ± 3.992621801525448e-31
ℓ = 3 : γ = 0.1837287415576813446704751844756020849388242272706860608187456557612265852219453 ± 1.917620514432176e-31
ℓ = 4 : γ = 0.1835136287278182764518617058558613428070617168658978878040179544815704040846418 ± 3.9126819515870045e-31
ℓ = 5 : γ = 0.1834932460279544338097548335430004911901572962747874965993780293630688252305876 ± 2.649601811295023e-31
ℓ = 6 : γ = 0.1834913090550747309459197780907288180521769769690369714475336404248443807802533 ± 2.8389561192793544e-31
ℓ = 7 : γ = 0.1834911247555301978698738155229523280714996129996220500013316285240991927271392 ± 2.076267885248325e-31
ℓ = 8 : γ = 0.1834911072154599688377776892634855783566668960062404491320423038564475092115929 ± 3.2784459490647463e-31
ℓ = 9 : γ = 0.18349110554607966257673462480819452401

## High-precision solution of rate LMI
First use ℓ=1 and perform high-accuracy bisection.
Then, use ℓ=10 and perturb the solution (make r a bit smaller) to certify that no improvement is possible

In [30]:
# initialize solver using good guess upper and lower bounds for r
bsmin( ρ -> solver_helper(α,β,η,m,L,ρ,1, norm="rho", bal=false, solver=Hypatia, T=BigFloat),
    big"0.92793311096382285", big"0.927933110963822861"; tol=big"1e-31", verbose=true, T=BigFloat)

iteration 1: (9.27933110963822850000000000000000031e-01, 9.27933110963822855499999999999996958e-01), gap = 5.5e-18
iteration 2: (9.27933110963822850000000000000000031e-01, 9.27933110963822852750000000000001576e-01), gap = 2.75e-18
iteration 3: (9.27933110963822850000000000000000031e-01, 9.27933110963822851374999999999997722e-01), gap = 1.375e-18
iteration 4: (9.27933110963822850000000000000000031e-01, 9.27933110963822850687499999999998876e-01), gap = 6.875e-19
iteration 5: (9.27933110963822850343749999999999454e-01, 9.27933110963822850687499999999998876e-01), gap = 3.4375e-19
iteration 6: (9.27933110963822850515624999999999165e-01, 9.27933110963822850687499999999998876e-01), gap = 1.71875e-19
iteration 7: (9.27933110963822850515624999999999165e-01, 9.27933110963822850601562500000002102e-01), gap = 8.59375e-20
iteration 8: (9.27933110963822850515624999999999165e-01, 9.27933110963822850558593749999997552e-01), gap = 4.296875e-20
iteration 9: (9.2793311096382285053710937500000144e-01, 9.2

0.9279331109638228505504059712948006222177355084568262100219726562499999999999959

In [8]:
# upper bound on r (certified with ℓ=1)
solver_helper(α, β, η, m, L, big"0.927933110963822850550405971295", 1; bal=false, returnsol=false, norm="rho", solver=Hypatia, T=BigFloat)

true

In [ ]:
# lower bound bound on r (certified with ℓ=10)

In [7]:
solver_helper(α, β, η, m, L, big"0.927933110963822850550405971294", 10; bal=false, returnsol=false, norm="rho", solver=Hypatia, T=BigFloat)

false